### Importando as Bibliotecas e Funções

In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import date, timedelta
import requests
import io

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df

### Consultando a API

In [ ]:
url = 'https://www.datafoot.org/api/matches/'
token = 'TOKEN'

headers = {
    'Authorization': f'Token {token}'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    base = pd.DataFrame(data)
else:
    print("Erro ao acessar a API:", response.status_code)

### Análise Exploratória dos Dados

In [ ]:
df = base.copy()
display(df)

,id,div,div_original_name,div_rank,date,home_team,away_team,fthg,ftag,hthg,htag,ftr,oddh_op,oddd_op,odda_op,oddh,oddd,odda,ah,oddahh,oddaha,ah_op,oddahh_op,oddaha_op,ah_ou,oddah_over,oddah_under,ah_ou_op,oddah_over_op,oddah_under_op,plh,pld,pla,plh_op,pld_op,pla_op,pl_ahh,pl_aha,pl_ahh_op,pl_aha_op,pl_ah_over,pl_ah_under,pl_ah_over_op,pl_ah_under_op,over_score,over_score_op,id_jogo
0,60304050,Iraq League,Iraq League,4 - Várzea,2021-01-01T08:00:00Z,Al Hudod,Al Najaf,2.0,1.0,1.0,1.0,H,2.600,2.875,2.625,2.60,2.875,2.625,0.00,1.875,1.925,0.00,1.875,1.925,1.75,1.85,1.95,1.75,1.850,1.950,1.6,-1.00,-1.00,1.6,-1.0,-1.0,0.875,-1.000,0.875,-1.00,0.85,-1.00,0.8500,-1.00,0.208835,0.208835,3112877
1,60304051,Iraq League,Iraq League,4 - Várzea,2021-01-01T08:00:00Z,Al Samawa SC,Al Kahrabaa,1.0,0.0,1.0,0.0,H,3.400,2.750,2.200,3.40,2.750,2.200,0.25,1.850,1.950,0.25,1.850,1.950,1.75,1.85,1.95,1.75,1.850,1.950,2.4,-1.00,-1.00,2.4,-1.0,-1.0,0.850,-1.000,0.850,-1.00,-1.00,0.95,-1.0000,0.95,0.208835,0.208835,3111477
2,60304052,Saudi Arabia Division 1,Saudi Arabia Division 1,4 - Várzea,2021-01-01T09:00:00Z,Al Khaleej Saihat,Al Jabalain,1.0,1.0,NaN,NaN,D,3.000,3.300,2.100,2.55,3.250,2.450,0.00,1.950,1.850,0.25,1.900,1.900,2.00,2.00,1.80,2.25,1.900,1.900,-1.0,2.25,-1.00,-1.0,2.3,-1.0,0.000,0.000,0.450,-0.50,0.00,0.00,-0.5000,0.45,0.252975,0.310371,2868277
3,60304053,Saudi Arabia Pro League,Saudi Arabia Pro League,2 - Destaque,2021-01-01T09:45:00Z,Damac FC,Al Fateh SC,1.0,2.0,0.0,1.0,A,2.300,3.400,2.600,1.95,3.500,3.400,-0.25,1.825,1.975,-0.25,1.950,1.850,2.75,1.80,2.00,2.75,1.775,2.025,-1.0,-1.00,2.40,-1.0,-1.0,1.6,-1.000,0.975,-1.000,0.85,0.40,-0.50,0.3875,-0.50,0.406584,0.407968,2825668
4,60304054,Saudi Arabia Pro League,Saudi Arabia Pro League,2 - Destaque,2021-01-01T09:50:00Z,Al Ain FC,Abha,2.0,4.0,1.0,1.0,A,1.909,3.500,3.300,2.10,3.200,3.250,-0.25,1.950,1.850,-0.25,1.950,1.850,2.25,1.85,1.95,2.75,1.950,1.850,-1.0,-1.00,2.25,-1.0,-1.0,2.3,-1.000,0.850,-1.000,0.85,0.85,-1.00,0.9500,-1.00,0.313556,0.398221,2825670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181556,60485606,France Ligue 1,France Ligue 1,1 - Elite,2024-11-09T17:00:00Z,Angers,PSG,NaN,NaN,NaN,NaN,,9.000,6.000,1.270,11.00,6.000,1.250,1.75,2.010,1.890,1.75,1.930,1.970,3.25,1.86,2.04,3.25,1.890,2.010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.485618,0.484164,8383584
181557,60485607,Brazil Serie A,Brazil Serie A,2 - Destaque,2024-11-09T19:00:00Z,Cruzeiro,Criciuma,NaN,NaN,NaN,NaN,,1.600,3.750,6.000,1.60,3.750,6.000,-1.00,2.090,1.810,-1.00,2.090,1.810,2.50,2.05,1.85,2.50,2.050,1.850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.351384,0.351384,8119943
181558,60485608,Brazil Serie A,Brazil Serie A,2 - Destaque,2024-11-09T19:00:00Z,Atletico GO,Bragantino,NaN,NaN,NaN,NaN,,2.250,3.400,3.100,2.25,3.400,3.100,-0.25,1.990,1.910,-0.25,1.990,1.910,2.25,2.00,1.90,2.25,2.000,1.900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.307252,0.307252,8169116
181559,60485609,Brazil Serie A,Brazil Serie A,2 - Destaque,2024-11-09T19:00:00Z,Fortaleza,Vasco da Gama,NaN,NaN,NaN,NaN,,1.900,3.500,4.000,1.90,3.500,4.000,-0.50,1.930,1.970,-0.50,1.930,1.970,2.25,1.97,1.93,2.25,1.970,1.930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.309125,0.309125,8170393


In [ ]:
df['date'] = pd.to_datetime(df['date'])

df['Date'] = df['date'].dt.date
df['Time'] = df['date'].dt.time

df = df.drop(columns=['date'])

df = df[['div_original_name','Date','Time','home_team','away_team',
         'hthg','htag','fthg','ftag','ftr',
         'oddh','oddd','odda',
         'ah','oddahh','oddaha',
         'ah_ou','oddah_over','oddah_under']]

df.columns = ['League','Date','Time','Home','Away',
         'Goals_H_HT','Goals_A_HT','Goals_H_FT','Goals_A_FT','Result',
         'Odd_H','Odd_D','Odd_A',
         'Asian_Handicap_MO','Odd_AH_H','Odd_AH_A',
         'Asian_Handicap_OU','Odd_AH_Over','Odd_AH_Under']

df[['Goals_H_HT', 'Goals_A_HT']] = df[['Goals_H_HT', 'Goals_A_HT']].fillna(0)

display(df)

,League,Date,Time,Home,Away,Goals_H_HT,Goals_A_HT,Goals_H_FT,Goals_A_FT,Result,Odd_H,Odd_D,Odd_A,Asian_Handicap_MO,Odd_AH_H,Odd_AH_A,Asian_Handicap_OU,Odd_AH_Over,Odd_AH_Under
0,Iraq League,2021-01-01,08:00:00,Al Hudod,Al Najaf,1.0,1.0,2.0,1.0,H,2.60,2.875,2.625,0.00,1.875,1.925,1.75,1.85,1.95
1,Iraq League,2021-01-01,08:00:00,Al Samawa SC,Al Kahrabaa,1.0,0.0,1.0,0.0,H,3.40,2.750,2.200,0.25,1.850,1.950,1.75,1.85,1.95
2,Saudi Arabia Division 1,2021-01-01,09:00:00,Al Khaleej Saihat,Al Jabalain,0.0,0.0,1.0,1.0,D,2.55,3.250,2.450,0.00,1.950,1.850,2.00,2.00,1.80
3,Saudi Arabia Pro League,2021-01-01,09:45:00,Damac FC,Al Fateh SC,0.0,1.0,1.0,2.0,A,1.95,3.500,3.400,-0.25,1.825,1.975,2.75,1.80,2.00
4,Saudi Arabia Pro League,2021-01-01,09:50:00,Al Ain FC,Abha,1.0,1.0,2.0,4.0,A,2.10,3.200,3.250,-0.25,1.950,1.850,2.25,1.85,1.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181556,France Ligue 1,2024-11-09,17:00:00,Angers,PSG,0.0,0.0,NaN,NaN,,11.00,6.000,1.250,1.75,2.010,1.890,3.25,1.86,2.04
181557,Brazil Serie A,2024-11-09,19:00:00,Cruzeiro,Criciuma,0.0,0.0,NaN,NaN,,1.60,3.750,6.000,-1.00,2.090,1.810,2.50,2.05,1.85
181558,Brazil Serie A,2024-11-09,19:00:00,Atletico GO,Bragantino,0.0,0.0,NaN,NaN,,2.25,3.400,3.100,-0.25,1.990,1.910,2.25,2.00,1.90
181559,Brazil Serie A,2024-11-09,19:00:00,Fortaleza,Vasco da Gama,0.0,0.0,NaN,NaN,,1.90,3.500,4.000,-0.50,1.930,1.970,2.25,1.97,1.93


In [ ]:
analysis_date = date.today() - timedelta(days=0)
print(analysis_date)

2024-11-02


In [ ]:
flt = df.Date < analysis_date
df0 = df[flt]
df0 = drop_reset_index(df0)
df0

,League,Date,Time,Home,Away,Goals_H_HT,Goals_A_HT,Goals_H_FT,Goals_A_FT,Result,Odd_H,Odd_D,Odd_A,Asian_Handicap_MO,Odd_AH_H,Odd_AH_A,Asian_Handicap_OU,Odd_AH_Over,Odd_AH_Under
1,Iraq League,2021-01-01,08:00:00,Al Hudod,Al Najaf,1.0,1.0,2.0,1.0,H,2.600,2.875,2.625,0.00,1.875,1.925,1.75,1.850,1.950
2,Iraq League,2021-01-01,08:00:00,Al Samawa SC,Al Kahrabaa,1.0,0.0,1.0,0.0,H,3.400,2.750,2.200,0.25,1.850,1.950,1.75,1.850,1.950
3,Saudi Arabia Division 1,2021-01-01,09:00:00,Al Khaleej Saihat,Al Jabalain,0.0,0.0,1.0,1.0,D,2.550,3.250,2.450,0.00,1.950,1.850,2.00,2.000,1.800
4,Saudi Arabia Pro League,2021-01-01,09:45:00,Damac FC,Al Fateh SC,0.0,1.0,1.0,2.0,A,1.950,3.500,3.400,-0.25,1.825,1.975,2.75,1.800,2.000
5,Saudi Arabia Pro League,2021-01-01,09:50:00,Al Ain FC,Abha,1.0,1.0,2.0,4.0,A,2.100,3.200,3.250,-0.25,1.950,1.850,2.25,1.850,1.950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180607,Mexico Liga de Expansion,2024-11-01,22:00:00,Tapatio,Club Celaya,1.0,0.0,1.0,1.0,D,2.625,3.300,2.625,0.00,1.875,1.925,2.50,1.975,1.825
180608,Mexico Liga de Expansion,2024-11-01,22:00:00,Venados FC,Universidad Guadalajara,1.0,0.0,3.0,0.0,H,2.250,3.300,3.100,-0.25,1.975,1.825,2.50,1.875,1.925
180609,Argentina Liga Profesional,2024-11-01,22:00:00,Argentinos Jrs,Velez Sarsfield,0.0,0.0,1.0,1.0,D,3.500,3.100,2.250,0.25,1.925,1.925,2.25,2.050,1.800
180610,Costa Rica Primera Division,2024-11-01,23:00:00,Municipal Liberia,Sporting San Jose,0.0,1.0,1.0,1.0,D,1.727,3.700,4.750,-0.75,1.925,1.875,2.75,1.925,1.875


In [ ]:
df0.to_csv("Base_DataFoot.csv", index=False)

In [ ]:
flt = df.Date >= analysis_date
df0 = df[flt]
# df0 = drop_reset_index(df0)
df0

,League,Date,Time,Home,Away,Goals_H_HT,Goals_A_HT,Goals_H_FT,Goals_A_FT,Result,Odd_H,Odd_D,Odd_A,Asian_Handicap_MO,Odd_AH_H,Odd_AH_A,Asian_Handicap_OU,Odd_AH_Over,Odd_AH_Under
180611,Mexico Liga de Expansion,2024-11-02,00:00:00,Club Atletico La Paz,Cancun FC,0.0,1.0,1.0,1.0,D,3.75,4.10,1.800,0.50,1.975,1.825,2.75,1.875,1.925
180612,Mexico Liga MX,2024-11-02,00:00:00,Mazatlan FC,Club America,0.0,4.0,0.0,5.0,A,3.50,3.60,2.100,0.25,2.050,1.800,2.50,1.825,2.025
180613,Mexico Liga MX,2024-11-02,00:05:00,Tijuana,Tigres UANL,0.0,1.0,0.0,3.0,A,3.30,3.70,2.050,0.25,2.025,1.825,2.75,1.825,2.025
180614,Australia ALeague,2024-11-02,01:00:00,Wellington Phoenix,Auckland FC,0.0,0.0,0.0,2.0,A,2.55,3.50,2.625,0.00,1.875,1.975,2.50,1.875,1.975
180615,Japan Football League,2024-11-02,01:00:00,Tochigi City,Okinawa SV,2.0,2.0,4.0,3.0,H,1.60,3.60,4.750,-0.75,1.800,2.000,3.00,1.850,1.950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181556,France Ligue 1,2024-11-09,17:00:00,Angers,PSG,0.0,0.0,NaN,NaN,,11.00,6.00,1.250,1.75,2.010,1.890,3.25,1.860,2.040
181557,Brazil Serie A,2024-11-09,19:00:00,Cruzeiro,Criciuma,0.0,0.0,NaN,NaN,,1.60,3.75,6.000,-1.00,2.090,1.810,2.50,2.050,1.850
181558,Brazil Serie A,2024-11-09,19:00:00,Atletico GO,Bragantino,0.0,0.0,NaN,NaN,,2.25,3.40,3.100,-0.25,1.990,1.910,2.25,2.000,1.900
181559,Brazil Serie A,2024-11-09,19:00:00,Fortaleza,Vasco da Gama,0.0,0.0,NaN,NaN,,1.90,3.50,4.000,-0.50,1.930,1.970,2.25,1.970,1.930
